# Merge the Slave Trade Profits Database

This Jupyter Notebook is intended to preprocess and merge the individual venture and transactions databases of the participating researchers into two CSVs: one for ventures and one for transactions. It uses [pandas](https://pandas.pydata.org/) for any data processing. 

In [ ]:
# Imports and helper functions
import pandas as pd
import re
import unicodedata
from pathlib import Path

def normalize_column_names(colname: str, maxlen: int = 32) -> str:
    
    # The following lines remove accents in the string
    s = unicodedata.normalize("NFKD", colname)
    s = ''.join(c for c in s if not unicodedata.combining(c))

    # The following lines make the string lowercase and replaces non-alphanumeric chars (like spaces) with underscore
    s = re.sub(r'[^a-zA-Z0-9_]', '_', s.lower())
    s = re.sub(r'_+', '_', s).strip('_')

    # If the result is empty or starts with digit, prepend with a 'v'
    if not s or s[0].isdigit():
        s = f"v{s}"
    return s[:maxlen]


def normalize_and_collapse(df: pd.DataFrame) -> pd.DataFrame:

    # Normalize the column names using the previously defined function
    normal_map = {c: normalize_column_names(c) for c in df.columns}
    df = df.rename(columns=normal_map)

    # Handle duplicates produced by the rename
    duplicates = df.columns[df.columns.duplicated()].unique()
    for dup in duplicates:
        cols = df.loc[:, dup]

        if cols.shape[1] > 1:
            combined = cols.bfill(axis=1).iloc[:, 0]
            df[dup] = combined

    df = df.loc[:, ~df.columns.duplicated()]
    return df

def remove_dots(df: pd.DataFrame) -> pd.DataFrame:
    """
    If a cell only has a dot in it, remove it and replace with <NA> (empty)
    """
    return df.replace('.', pd.NA)

In [3]:
# Define paths to input files
DATA_DIR      = Path("../data")  
OUTPUT_DIR    = Path("../data")
OUTPUT_DIR.mkdir(exist_ok=True)

cash_flow_files = [
    DATA_DIR / "Cash flow database DR.csv",
    DATA_DIR / "Cash flow database GD.csv",
    DATA_DIR / "Cash flow database GK.csv",
    DATA_DIR / "Cash flow database KR - new.csv",
    DATA_DIR / "Cash flow database MR - new.csv",
]

venture_files = [
    DATA_DIR / "Venture database DR.csv",
    DATA_DIR / "Venture database GD.csv",
    DATA_DIR / "Venture database GK.csv",
    DATA_DIR / "Venture database KR - new.csv",
    DATA_DIR / "Venture database MR.csv"
]

Let's begin with merging the cashflow files (to be renamed transactions).

In [5]:
cash_flow_dfs = []

# Read all individual files into a list
for path in cash_flow_files:
    df = pd.read_csv(path, encoding="utf-8")

    # Normalize column names
    df = normalize_and_collapse(remove_dots(df))

    # Make meansofpaymentreturn and dateoftransaction strings to prevent type mismatch in pandas (doesn't matter for CSV anyway)
    for col in ['meansofpaymentreturn', 'dateoftransaction']:
        if col in df.columns:
            df[col] = df[col].astype(str)

    # Convert values in the value column to numeric
    if 'value' in df.columns:
        df['value'] = (
            df['value'].astype(str).str.replace(',', '.', regex=False)
            .replace('', pd.NA)
        )
        df['value'] = pd.to_numeric(df['value'], errors='coerce')

    # Remove rows without a venture id (this is a required key into the ventures file)
    if 'ventureid' in df.columns:
        df = df[df['ventureid'].notna()]

    cash_flow_dfs.append(df)

# Concatenate all individual dataframes into one
cash_flow_all = pd.concat(cash_flow_dfs, ignore_index=True)

# Add categories from specification_categories.csv (this works if that csv is actually tsv)
def read_spec_categories(path: Path) -> pd.DataFrame:

    sample = Path(path).read_text(encoding="utf-8", errors="replace").splitlines()[0]
    if sample.count("\t") >= 1:
        sep = "\t"
    elif sample.count(";") >= 1 and sample.count(",") == 0:
        sep = ";"
    else:
        sep = ","
    return pd.read_csv(path, sep=sep, encoding="utf-8")

spec_path = DATA_DIR / "specification_categories.csv"
spec_df   = read_spec_categories(spec_path)

spec_df = normalize_and_collapse(remove_dots(spec_df))
spec_df = (
    spec_df[['specification', 'specificationcategory']]
    .drop_duplicates(subset='specification', keep='first')
)

# Many to one one merge
cash_flow_all = cash_flow_all.merge(
    spec_df,
    on='specification',
    how='left',
    validate='m:1'
)

assert cash_flow_all['ventureid'].isna().sum() == 0

cf_out = "transactions.csv"
cash_flow_all.to_csv(cf_out, index=False, float_format="%.8f")

Now let's process the venture files.

In [7]:
venture_dfs = []

# Read all individual files into a list
for path in venture_files:
    df = pd.read_csv(path, encoding="utf-8")
    df = normalize_and_collapse(remove_dots(df))

    # Make some columns strings to prevent pandas from misinterpreting them
    for col in df.columns:
        if col.startswith(('date', 'place', 'number', 'voyageidintstd',
                           'internalcrossref', 'nameofthecaptain',
                           'profitsreportedinsource')):
            df[col] = df[col].astype(str)

    # Make shareoftheship and numberofvoyages numeric
    if 'shareoftheship' in df.columns:
        df['shareoftheship'] = (
            df['shareoftheship']
            .astype(str)
            .str.replace(',', '.', regex=False)
            .replace('', pd.NA)
        )
        df['shareoftheship'] = pd.to_numeric(
            df['shareoftheship'], errors='coerce')

    if 'numberofvoyages' in df.columns:
        df['numberofvoyages'] = pd.to_numeric(
            df['numberofvoyages'], errors='coerce')

    # Rename fate to fatecol if needed (remnant from some legacy template?)
    if 'fate' in df.columns and 'fatecol' not in df.columns:
        df = df.rename(columns={'fate': 'fatecol'})

    venture_dfs.append(df)

# Merge all dfs
venture_all = pd.concat(venture_dfs, ignore_index=True)

# Extra standardization steps
if 'perspectiveofsource' in venture_all.columns:
    venture_all['perspectiveofsource'] = venture_all[
        'perspectiveofsource'
    ].replace({'investor': 'Investor', 'Owner?': 'Owner'}) # investor and owner

if 'completedataonoutlays' in venture_all.columns:
    venture_all['completedataonoutlays'] = venture_all[
        'completedataonoutlays'
    ].str.replace(r'(?i)^n$', 'no', regex=True) # replace n with no

# Remove dots from names --> SHOULD WE KEEP THIS?
for col in ['internalcrossref', 'nameoftheoutfitter', 'nameofthecaptain']:
    if col in venture_all.columns:
        venture_all[col] = venture_all[col].replace('.', '', regex=False)

# Make sure slave counts are numeric
for col in ['numberofslavespurchased', 'numberofslavessold']:
    if col in venture_all.columns:
        venture_all[col] = pd.to_numeric(
            venture_all[col], errors='coerce')

venture_out = "venture all.csv"
venture_all.to_csv(venture_out, index=False)